In [1]:
import logging
import torch
import pandas as pd
import shutil

/Users/jonas/Projects/others/twitter-sentiment-classification/venv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from bert import train
from train_test_split import select_train_with_cluster, select_eval_with_cluster

[nltk_data] Downloading package stopwords to /Users/jonas/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/jonas/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/jonas/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
logging.basicConfig(level=logging.INFO)

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

---

In [5]:
MODEL = 'distilbert-base-uncased' # 'cardiffnlp/twitter-roberta-base-sentiment-latest'
TOKENIZER = 'bert-base-uncased'

EPOCHS = 1
BATCH_SIZE = 32

In [10]:
df_cluster_map_train = pd.read_csv('clustering+bert/train.csv')[['index', 'cluster']]
df_cluster_map_eval = pd.read_csv('clustering+bert/eval.csv')[['index', 'cluster']]
df_cluster_map = pd.concat([df_cluster_map_train, df_cluster_map_eval])
df_cluster_map

,index,cluster
0,157049.0,1
1,2366208.0,2
2,1948945.0,0
3,1684769.0,5
4,2262152.0,1
...,...,...
1249995,1478680.0,2
1249996,1972646.0,4
1249997,1710597.0,5
1249998,1835784.0,4


In [11]:
CLUSTERS = df_cluster_map['cluster'].unique()
CLUSTERS

array([1, 2, 0, 5, 3, 4, 6])

In [12]:
def train_cluster(cluster: int) -> float:
  print(f'====== TRAINING ON CLUSTER {cluster} ======')
  df_train = select_train_with_cluster(df_cluster_map, cluster)
  df_eval = select_eval_with_cluster(df_cluster_map, cluster)
  
  try:
    shutil.rmtree('cardiffnlp')
  except OSError:
    pass

  model = train(MODEL, TOKENIZER, device, df_train, df_eval, batch_size=BATCH_SIZE, epochs=EPOCHS)
  return model

In [13]:
for cluster in CLUSTERS:
  model = train_cluster(cluster)
  model.save_pretrained(f'cluster-{cluster}')

====== TRAINING ON CLUSTER 1 ======


Casting the dataset: 100%|██████████| 4/4 [00:00<00:00, 77.04ba/s]
